I couldn't crawl glassdoor data in a normal way, like by BeautifulSoup. 
Glassdoor doesn't allow bots to crawl data, and access to the specified resource is forbidden.

I referred to "https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb".

1. data sientist jobs in the US (using 30 urls)
url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17.htm'
url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17_IP2.htm'
:
url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17_IP30.htm'

2. software enginner jobs in the US (using 30 urls)
url = 'https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20.htm'
url = "https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20_IP2.htm"
:
url = "https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20_IP30.htm"

In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
from datetime import datetime
import pandas as pd

In [9]:
def get_jobs(verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    #url = 'https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20.htm'
    #url = "https://www.glassdoor.com/Job/us-software-engineer-jobs-SRCH_IL.0,2_IN1_KO3,20_IP2.htm"

    url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17.htm'
    #url = 'https://www.glassdoor.com/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17_IP2.htm'
    
    driver.get(url)
    jobs = []

    #Let the page load. Change this number based on your internet speed.
    #Or, wait until the webpage is loaded, instead of hardcoding it.
    time.sleep(4)

    #Test for the "Sign Up" prompt and get rid of it.
    try:
        driver.find_element_by_class_name("selected").click()
    except ElementClickInterceptedException:
        pass

    time.sleep(3)

    try:
        driver.find_element_by_class_name("ModalStyle__xBtn___29PT9").click()  #clicking to the X.
    except NoSuchElementException:
        pass

    timestamp = datetime.now().isoformat()

    #Going through each job in this page
    job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
    time.sleep(.5)
    for job_button in job_buttons:  

        print("Progress: {}".format("" + str(len(jobs))))

        job_button.click()  #You might 
        time.sleep(1)
        collected_successfully = False

        while not collected_successfully:
            try:
                company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                location = driver.find_element_by_xpath('.//div[@class="location"]').text
                job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                collected_successfully = True
            except:
                time.sleep(5)

        time.sleep(0.1)
        try:
            salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
        except NoSuchElementException:
            salary_estimate = -1 #You need to set a "not found value. It's important."

        try:
            rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
        except NoSuchElementException:
            rating = -1 #You need to set a "not found value. It's important."

        #Printing for debugging
        if verbose:
            print("Job Title: {}".format(job_title))
            print("Salary Estimate: {}".format(salary_estimate))
            print("Job Description: {}".format(job_description[:500]))
            print("Rating: {}".format(rating))
            print("Company Name: {}".format(company_name))
            print("Location: {}".format(location))

        #Going to the Company tab...
        #clicking on this:
        #<div class="tab" data-tab-type="overview"><span>Company</span></div>
        time.sleep(0.5)
        try:
            driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

            try:
                #<div class="infoEntity">
                #    <label>Headquarters</label>
                #    <span class="value">San Francisco, CA</span>
                #</div>
                headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
            except NoSuchElementException:
                headquarters = -1

            try:
                size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
            except NoSuchElementException:
                size = -1

            try:
                founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
            except NoSuchElementException:
                industry = -1

            try:
                sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
            except NoSuchElementException:
                sector = -1

            try:
                revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
            except NoSuchElementException:
                revenue = -1

            try:
                competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
            except NoSuchElementException:
                competitors = -1
            
            try:
                link = driver.find_element_by_xpath('//*[(@id = "EmpBasicInfo")]//*[contains(concat( " ", @class, " " ), concat( " ", "link", " " ))]')
                url2 = link.get_attribute('href')
            except NoSuchElementException:
                url2 = -1                

        except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
            headquarters = -1
            size = -1
            founded = -1
            type_of_ownership = -1
            industry = -1
            sector = -1
            revenue = -1
            competitors = -1


        if verbose:
            print("Headquarters: {}".format(headquarters))
            print("Size: {}".format(size))
            print("Founded: {}".format(founded))
            print("Type of Ownership: {}".format(type_of_ownership))
            print("Industry: {}".format(industry))
            print("Sector: {}".format(sector))
            print("Revenue: {}".format(revenue))
            print("Competitors: {}".format(competitors))
            print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

        jobs.append({"Job Title" : job_title,
        "Salary Estimate" : salary_estimate,
        "Job Description" : job_description,
        "Rating" : rating,
        "Company Name" : company_name,
        "Location" : location,
        "Headquarters" : headquarters,
        "Size" : size,
        "Founded" : founded,
        "Type of ownership" : type_of_ownership,
        "Industry" : industry,
        "Sector" : sector,
        "Revenue" : revenue,
        "Competitors" : competitors,
        "url": url2,
        "timestamp" : timestamp})
        #add job to jobs


    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [10]:
#This line will open a new chrome window and start the scraping.
df = get_jobs(False)
df

Progress: 0
Progress: 1
Progress: 2
Progress: 3
Progress: 4
Progress: 5
Progress: 6
Progress: 7
Progress: 8
Progress: 9
Progress: 10
Progress: 11
Progress: 12
Progress: 13
Progress: 14
Progress: 15
Progress: 16
Progress: 17
Progress: 18
Progress: 19
Progress: 20
Progress: 21
Progress: 22
Progress: 23
Progress: 24
Progress: 25
Progress: 26
Progress: 27
Progress: 28
Progress: 29


,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership,timestamp,url
0,Amrock\n3.6,-1,1997,"Detroit, MI",Real Estate,Preferred Qualifications\nMasters degree or do...,Data Scientist,"Detroit, MI",3.6,$500 million to $1 billion (USD),-1,Real Estate,1001 to 5000 employees,Company - Private,2020-04-16T21:55:11.204721,http://www.amrock.com/
1,ManTech\n4.1,-1,1968,"Herndon, VA",Research & Development,"Secure our Nation, Ignite your Future\n\nSumma...",Data Scientist,"Chantilly, VA",4.1,$1 to $2 billion (USD),-1,Business Services,5001 to 10000 employees,Company - Public,2020-04-16T21:55:11.204721,http://www.mantech.com/
2,Smule\n4.2,-1,2008,"San Francisco, CA",Internet,The Smule mission is to connect people all ove...,Machine Learning Engineer,"San Francisco, CA",4.2,$100 to $500 million (USD),-1,Information Technology,51 to 200 employees,Company - Private,2020-04-16T21:55:11.204721,http://www.smule.com/
3,CyrusOne\n3.5,"Digital Realty, CoreSite, Equinix",2000,"Dallas, TX",Real Estate,CyrusOne is seeking a talented Data Scientist ...,Data Scientist,"Dallas, TX",3.5,$1 to $2 billion (USD),-1,Real Estate,201 to 500 employees,Company - Public,2020-04-16T21:55:11.204721,http://www.cyrusone.com/
4,ClearOne Advantage\n4.1,-1,2008,"Baltimore, MD",Banks & Credit Unions,Job Description\n\n**Please only local candida...,Data Scientist,"Baltimore, MD",4.1,Unknown / Non-Applicable,-1,Finance,501 to 1000 employees,Company - Private,2020-04-16T21:55:11.204721,http://www.clearoneadvantage.com/
5,Miracle Software Systems\n3.3,"CSC, IBM",1994,"Novi, MI",IT Services,Position Description: The Analytics Research a...,Data Scientists,Nebraska,3.3,$50 to $100 million (USD),-1,Information Technology,51 to 200 employees,Company - Private,2020-04-16T21:55:11.204721,-1
6,Parkland Center for Clinical Innovation\n4.2,-1,2012,"Dallas, TX",IT Services,"Looking for your next opportunity? If so, choo...",Data Engineer - PCCI,"Dallas, TX",4.2,Unknown / Non-Applicable,-1,Information Technology,1 to 50 employees,Nonprofit Organization,2020-04-16T21:55:11.204721,http://www.pccinnovation.org/
7,PNNL\n3.8,"Oak Ridge National Laboratory, National Renewa...",1965,"Richland, WA",Energy,*Organization and Job ID**\nJob ID: 310677\n\n...,Data Scientist,"Richland, WA",3.8,$500 million to $1 billion (USD),-1,"Oil, Gas, Energy & Utilities",1001 to 5000 employees,Government,2020-04-16T21:55:11.204721,-1
8,Mars\n3.9,-1,1911,"Mc Lean, VA",Food & Beverage Manufacturing,JOB CATEGORY:\n\nInformation Services\n\nREQUI...,Data Scientist,Oregon,3.9,$10+ billion (USD),-1,Manufacturing,10000+ employees,Company - Private,2020-04-16T21:55:11.204721,http://www.mars.com/
9,IFG Companies\n2.9,"Colony Specialty, Markel, RLI",1985,"Hartford, CT",Insurance Carriers,Job Brief\n\nThe ideal candidate will have pre...,Data Scientist,"New York, NY",2.9,Unknown / Non-Applicable,-1,Insurance,201 to 500 employees,Company - Private,2020-04-16T21:55:11.204721,http://www.ifgcompanies.com/


In [11]:
df.to_csv(r'dsdf1.csv', index = False, header=False)

In [ ]:
#df.to_csv(r'dsdf1.csv', index = False, header=True)